In [1]:
import numpy as np 
import pandas as pd

# Loading the multiple choices dataset, we will not look to the free form data on this study
mc = pd.read_csv('data/multipleChoiceResponses.csv', low_memory=False)

# Separating questions from answers
# This Series stores all questions
mcQ = mc.iloc[0,:]
# This DataFrame stores all answers
mcA = mc.iloc[1:,:]

In [2]:
mc.shape, mcA.shape

((23860, 395), (23859, 395))

In [3]:
# removing everyone that took less than 4 minutes or more than 600 minutes to answer the survey
less3 = mcA[round(mcA.iloc[:,0].astype(int) / 60) <= 4].index
mcA = mcA.drop(less3, axis=0)
more300 = mcA[round(mcA.iloc[:,0].astype(int) / 60) >= 600].index
mcA = mcA.drop(more300, axis=0)

# removing gender trolls, because we noticed from other kernels thata there are some ouliers here
gender_trolls = mcA[(mcA.Q1 == 'Prefer to self-describe') | (mcA.Q1 == 'Prefer not to say')].index
mcA = mcA.drop(list(gender_trolls), axis=0)

# removing student trolls, because a student won't make more than 250k a year.
student_trolls = mcA[((mcA.Q6 == 'Student') & (mcA.Q9 > '500,000+')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '400-500,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '300-400,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '250-300,000'))].index
mcA = mcA.drop(list(student_trolls), axis=0)

# dropping all NaN and I do not wish to disclose my approximate yearly compensation, because we are only interested in respondents that revealed their earnings
mcA = mcA[~mcA.Q9.isnull()].copy()
not_disclosed = mcA[mcA.Q9 == 'I do not wish to disclose my approximate yearly compensation'].index
mcA = mcA.drop(list(not_disclosed), axis=0)

In [4]:
# Remove all "OTHER_TEXT columns"
mcA = mcA[[c for c in mcA.columns if not c.endswith("_OTHER_TEXT") and c != 'Q32_OTHER']]

In [5]:
mcA.shape

(12956, 365)

In [6]:
q12_software_dummies = pd.get_dummies(mcA[['Q12_MULTIPLE_CHOICE']])  # Encode Q12

In [7]:
mcA = mcA[[c for c in mcA.columns if not c.endswith("TEXT")]]  # Remove all Q12_Part_*
mcA = mcA[[c for c in mcA.columns if c != "Q12_MULTIPLE_CHOICE"]]  # Remove Q12 itself
mcA = pd.concat([mcA, q12_software_dummies], axis=1)  # Append encoded Q12

In [8]:
minusone_cols = (mcA == '-1').sum(axis=0)
minusone_cols = minusone_cols[minusone_cols > 0]

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [9]:
for name, v in zip(minusone_cols.index, minusone_cols):
    print(name, v)

In [10]:
mcA.iloc[:5, 235:240]

,Q31_Part_11,Q31_Part_12,Q32,Q33_Part_1,Q33_Part_2
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Time Series Data,Government websites,NaN
5,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Numerical Data,NaN,NaN
8,NaN,NaN,Image Data,NaN,NaN


In [11]:
mcA.head(1)

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,"Q12_MULTIPLE_CHOICE_Advanced statistical software (SPSS, SAS, etc.)","Q12_MULTIPLE_CHOICE_Basic statistical software (Microsoft Excel, Google Sheets, etc.)","Q12_MULTIPLE_CHOICE_Business intelligence software (Salesforce, Tableau, Spotfire, etc.)","Q12_MULTIPLE_CHOICE_Cloud-based data software & APIs (AWS, GCP, Azure, etc.)","Q12_MULTIPLE_CHOICE_Local or hosted development environments (RStudio, JupyterLab, etc.)",Q12_MULTIPLE_CHOICE_Other
2,434,Male,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,Manufacturing/Fabrication,5-10,"10-20,000",...,NaN,NaN,NaN,NaN,0,1,0,0,0,0


In [12]:
descriptive_names = ['duration', 'sex', 'age', 'country', 'education',  'undergrad_major', 'role', 'industry', 'experience', 'salary'] 
mcA.columns = descriptive_names + list(mcA.columns[len(descriptive_names):])

In [19]:
cols = [column for column in mcA.columns if "Part" in column ]
def replace_vals(row):
    for col in cols:
        if pd.isna(row[col]):
            row[col] = 0
        elif row[col]:
            row[col] = 1
    return row
test = mcA.apply(replace_vals, axis=1)

In [46]:
na_columns = test.columns[pd.isna(test).sum() > 0]

In [47]:
pd.isna(test[na_columns]).sum()

undergrad_major      91
experience           61
Q10                  75
Q17                2836
Q18                 618
Q20                4180
Q22                4624
Q23                 737
Q24                 753
Q25                 772
Q26                 778
Q32                3571
Q37                6270
Q40                2300
Q43                3955
Q46                3797
Q48                3783
dtype: int64

In [55]:
test.shape

(12956, 365)

In [28]:
test[test.columns[pd.isna(test).sum() > 0]].shape

(12956, 17)

In [21]:
test.to_csv('transformed_v2.csv')